## Installing some of the dependencies 

In [111]:
! pip install psycopg2 
! pip install requests
! pip install SQLAlchemy

You should consider upgrading via the '/Users/apple/Desktop/tutorial/Postgresql_etl/myve/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/apple/Desktop/tutorial/Postgresql_etl/myve/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/apple/Desktop/tutorial/Postgresql_etl/myve/bin/python3 -m pip install --upgrade pip' command.


# Importing dependencies 

In [112]:
import pandas as pd
import numpy
import requests
import json
import csv

# Extraction Layer: Extracting the dataset from RapidAPI

In [113]:

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"2000"}

headers = {
	"x-rapidapi-key": "ce3693686amshd8a81ba62ce435ep1113afjsn137d2b9aa172",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# saving the data as a file

filename = 'propertyRecords.json'

# to write the file into the filename
with open (filename, 'w') as file:
    json.dump(data,file, indent = 4)

In [114]:
# to read into a dataframe

propertyrecords_df = pd.read_json('propertyRecords.json')


# Transforming the dataset

In [115]:
propertyrecords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   addressLine2      107 non-null    object 
 6   assessorID        372 non-null    object 
 7   county            499 non-null    object 
 8   legalDescription  359 non-null    object 
 9   squareFootage     392 non-null    float64
 10  subdivision       314 non-null    object 
 11  yearBuilt         368 non-null    float64
 12  zoning            207 non-null    object 
 13  bathrooms         372 non-null    float64
 14  lotSize           360 non-null    float64
 15  propertyType      413 non-null    object 
 16  ownerOccupied     320 non-null    float64
 1

In [116]:
# approach of cleaning
propertyrecords_df['features'] = propertyrecords_df['features'].apply(json.dumps)

# second approach to find and replace the nulls
propertyrecords_df.fillna({
        'bedrooms': 0,
        'addressLine2': 'Not available', 
        'squareFootage': 0,
        'yearBuilt': 0,
        'features': 'None',
        'assessorID': 'Unknown',
        'legalDescription': 'Not available',
        'subdivision': 'Not available', 
         'zoning': 'Unknown', 
         'bathrooms': 0, 
         'lotSize': 0,
         'propertyType': 'Unknown', 
         'taxAssessment': 'Not available',
        'propertyTaxes':  'Not available', 
         'lastSalePrice': 0,
        'lastSaleDate': 'Not available',
        'owner': 'Unknown',
        'ownerOccupied': 0,
        'addressLine2': 'Unknown',
        'county': 'Not available'}, inplace =True)

# to extract the id from the address
propertyrecords_df['id'] = propertyrecords_df['id'].apply(lambda x: id(x))
    
    

    



In [117]:
location_dim = propertyrecords_df[['addressLine2','county','zipCode','formattedAddress','state','city']].drop_duplicates().reset_index(drop=True)
location_dim['location_id'] = location_dim.index +1

propertyrecords_df = propertyrecords_df.merge(
    location_dim[['location_id', 'addressLine2','county','zipCode','formattedAddress','state','city']],  # Bring sales_id into propertyrecords_df
    on=['addressLine2','county','zipCode','formattedAddress','state','city'],  # Match on shared columns
    how='left'
)

In [118]:

features_dim = propertyrecords_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim['feature_id'] = features_dim.index +1  # Ensure consistent naming

# Merge features_dim into propertyrecords_df
propertyrecords_df = propertyrecords_df.merge(
    features_dim[['feature_id', 'features', 'propertyType', 'zoning']],  # Use 'feature_id' consistently
    on=['features', 'propertyType', 'zoning'],  # Match on shared columns
    how='left'
)


In [119]:
sales_fact = propertyrecords_df[['lastSalePrice', 'lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_fact['sales_id'] = sales_fact.index + 1 # Assign sales_id directly from the index

# Map sales_id back to propertyrecords_df
propertyrecords_df = propertyrecords_df.merge(
    sales_fact[['sales_id', 'lastSalePrice', 'lastSaleDate']],  # Bring sales_id into propertyrecords_df
    on=['lastSalePrice', 'lastSaleDate'],  # Match on shared columns
    how='left'
)

In [120]:
fact_columns = ['id', 'sales_id','feature_id','location_id','bedrooms', 'squareFootage','bathrooms', 'lotSize','lastSalePrice','lastSaleDate', 'longitude', 'latitude']
fact_table = propertyrecords_df[fact_columns]

In [121]:
sales_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lastSalePrice  311 non-null    float64
 1   lastSaleDate   311 non-null    object 
 2   sales_id       311 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 7.4+ KB


In [122]:
# saving dataset as csv
fact_table.to_csv('property_fact.csv', index=False)
location_dim.to_csv('location_dimension.csv', index = False)
sales_fact.to_csv('sales_facts.csv', index = False)
features_dim.to_csv('features_dimension.csv', index = False)

# Loading dataset to Postgresql

In [129]:
# Connecting to Postgresql

import psycopg2

In [146]:
def get_db_connection():
    try:
        connection = psycopg2.connect(
        host = 'localhost',
        port = 5432,
        user = 'postgres',
        password = 'chichi',
        database = 'Zapco_db')
        
        return connection
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None
    

conn = get_db_connection()
    

In [153]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Corrected SQL statements
    create_table_query = '''
        CREATE SCHEMA IF NOT EXISTS zapco_schema;

        DROP TABLE IF EXISTS zapco_schema.fact_table;
        DROP TABLE IF EXISTS zapco_schema.location_dim;
        DROP TABLE IF EXISTS zapco_schema.sales_facts;
        DROP TABLE IF EXISTS zapco_schema.features_dim;

        CREATE TABLE zapco_schema.fact_table (
            id NUMERIC, 
            sales_id NUMERIC,
            feature_id NUMERIC,
            location_id NUMERIC,
            bedrooms FLOAT, 
            squareFootage FLOAT,
            bathrooms FLOAT, 
            lotSize FLOAT,
            lastSalePrice FLOAT,
            lastSaleDate DATE, 
            longitude FLOAT, 
            latitude FLOAT
        );

        CREATE TABLE zapco_schema.location_dim (
            addressLine2 VARCHAR(300),
            county VARCHAR(300),
            zipCode INTEGER,
            formattedAddress VARCHAR(300),
            state VARCHAR(200),
            city VARCHAR(200),
            location_id INTEGER
        );

        CREATE TABLE zapco_schema.features_dim (
            features TEXT, 
            propertyType TEXT,
            zoning  TEXT,
            feature_id INT
        );

        CREATE TABLE zapco_schema.sales_facts (
            lastSalePrice NUMERIC, 
            lastSaleDate DATE,
            sales_id INT
        );
    '''
    
    # Execute query and commit changes
    try:
        cursor.execute(create_table_query)
        conn.commit()
        print("Tables created successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        cursor.close()
        conn.close()
        
create_tables()

Tables created successfully.


In [155]:

def load_data(csv_path, table_name, fact_columns):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            # Replace 'Not available' or empty string with None for lastSaleDate column
            row = [None if (cell == ' ' or cell == 'Not available') and col_name == 'lastSaleDate' else cell for cell, col_name in zip(row, fact_columns)]
            
            # Prepare placeholders for the insert statement
            placeholders = ', '.join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} ({", ".join(fact_columns)}) VALUES ({placeholders});'
            
            # Execute the query with the row data
            cursor.execute(query, row)

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection properly
    cursor.close()
    conn.close()

# Define the columns for the fact table (assuming these match your table schema)
fact_columns = ['id', 'sales_id', 'feature_id', 'location_id', 'bedrooms', 'squareFootage', 'bathrooms', 'lotSize', 'lastSalePrice', 'lastSaleDate', 'longitude', 'latitude']

# Load data for the fact table
fact_csv_path = 'property_fact.csv'
load_data(fact_csv_path, 'zapco_schema.fact_table', fact_columns)

# Load data for the features table
features_csv_path = 'features_dimension.csv'
load_data(features_csv_path, 'zapco_schema.features_dim', ['features', 'propertyType', 'zoning', 'feature_id'])

# Load data for the location table
location_csv_path = 'location_dimension.csv'
load_data(location_csv_path, 'zapco_schema.location_dim', ['addressLine2', 'county', 'zipCode', 'formattedAddress', 'state', 'city', 'location_id'])


In [157]:
sales_csv_path = 'sales_facts.csv'
load_data(sales_csv_path, 'zapco_schema.sales_facts',['lastSalePrice','lastSaleDate','sales_id'])